# Classification X-ray pneumonia images using Bigdl Approach

## Install Bigdl

In [ ]:
# uninstall pyspark, we install it with bigdl because of compatibility
!pip uninstall pyspark

Found existing installation: pyspark 3.5.5
Uninstalling pyspark-3.5.5:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-connect-shell
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
  

In [ ]:
pip install bigdl-orca-spark3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 20.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463458 sha256=dbad31168072ca6fd2cc689b766c2728c62e6369d7d023f3af47ada0af8546b2
  Stored in directory: /root/.cache/pip/wheels/25/da/89/3c1760252397d50554c2b3a66ab0ea57e1460fdab21d0aa968
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
   

## Start SparkContext

In [ ]:
from bigdl.orca import init_orca_context
init_orca_context(cluster_mode="local", memory="8g")

Initializing orca context
For Spark local mode, default to use all the cores on the node.
Current pyspark location is : /usr/local/lib/python3.11/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.11/dist-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.3-2.4.0-jar-with-dependencies.jar:/usr/local/lib/python3.11/dist-packages/bigdl/share/core/lib/all-2.4.0-20230420.050641-1.jar:/usr/local/lib/python3.11/dist-packages/bigdl/share/orca/lib/bigdl-orca-spark_3.1.3-2.4.0-jar-with-dependencies.jar pyspark-shell 
Successfully got a SparkContext


<SparkContext master=local[*] appName=pyspark-shell>

## Import Packages and Modules

In [ ]:
import os
import shutil
import zipfile

import tensorflow as tf
from tensorflow import keras


## Download and Extract Dataset

In [ ]:
def downlaod_extract_dataset():
  """
  This function downloads and extracts x-ray images from kaggle
  """

  # Create the .kaggle directory if it doesn't exist
  os.makedirs("/root/.kaggle", exist_ok=True)

  # Move the kaggle.json file
  shutil.move("kaggle.json", "/root/.kaggle/")

  # Set permissions
  os.chmod("/root/.kaggle/kaggle.json", 600)

  # Download kaggle dataset
  !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

  # Unzip the dataset
  with zipfile.ZipFile("/content/chest-xray-pneumonia.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

  print("Dataset extracted successfully!")


In [ ]:
downlaod_extract_dataset()

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:21<00:00, 242MB/s]
100% 2.29G/2.29G [00:21<00:00, 117MB/s]
Dataset extracted successfully!


## Split dataset into Train, Test, Val

In [ ]:
# Directories for train,val, test data
train_dir = "/content/dataset/chest_xray/train/"
val_dir = "/content/dataset/chest_xray/val/"
test_dir = "/content/dataset/chest_xray/test/"

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 224
CHANNEL = 1


train_data = keras.preprocessing.image_dataset_from_directory(
    directory = train_dir,
    image_size = (IMG_SIZE,IMG_SIZE),
    label_mode = 'binary',
    batch_size = BATCH_SIZE,
    shuffle = True
).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = keras.preprocessing.image_dataset_from_directory(
    directory = test_dir,
    image_size = (IMG_SIZE,IMG_SIZE),
    label_mode = 'binary',
    batch_size = BATCH_SIZE
).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

val_data = keras.preprocessing.image_dataset_from_directory(
    directory = val_dir,
    image_size = (IMG_SIZE,IMG_SIZE),
    label_mode = 'binary',
    batch_size = BATCH_SIZE
)

# The number of classes
class_names = val_data.class_names
val_data = val_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
print(f"Class Names: {class_names}")

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Class Names: ['NORMAL', 'PNEUMONIA']


## Define Model

In [ ]:
# Resize and Scaling Images
data_augmentation = keras.Sequential([
  keras.layers.Resizing(height=224, width=224),
  keras.layers.Rescaling(1./255)
], name ="data_augmentation")



model = keras.Sequential(
    [data_augmentation,
     keras.layers.Conv2D(20, kernel_size=(5, 5), strides=(1, 1), activation='relu',
                         padding='valid'),
     keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
     keras.layers.Conv2D(50, kernel_size=(5, 5), strides=(1, 1), activation='relu',
                         padding='valid'),
     keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
     keras.layers.Flatten(),
     keras.layers.Dense(500, activation='relu'),
     keras.layers.Dense(1, activation='sigmoid'),
     ]
)

model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Orca Estimator

In [ ]:
from bigdl.orca.learn.tf2.estimator import Estimator

In [ ]:
!pip install ray[default]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 34.3 MB/s eta 0:00:00


In [ ]:
est = Estimator.from_keras(keras_model=model)

2025-03-19 10:16:08,040	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://172.28.0.12:8265 


RayContext(dashboard_url='172.28.0.12:8265', python_version='3.11.11', ray_version='2.43.0', ray_commit='ecdcdc6a6e63dc4bcd6ea16aae256ce4d32a7e2c')


/usr/local/lib/python3.11/dist-packages/ray/_private/worker.py:1241: UserWarning: len(ctx) is deprecated. Use len(ctx.address_info) instead.
  warnings.warn("len(ctx) is deprecated. Use len(ctx.address_info) instead.")
(Worker pid=2139) 2025-03-19 10:16:14.488769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(Worker pid=2139) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(Worker pid=2139) E0000 00:00:1742379374.525150    2139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(Worker pid=2139) E0000 00:00:1742379374.538242    2139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(Worker pid=2139) WARNING:tensorflow:From /usr/local/lib/py

## Fit Estimator on train data

In [ ]:
est.fit(data=train_data,
        batch_size=BATCH_SIZE,
        epochs=30,
        validation_data=val_data)

## Evalute the Estimator

In [ ]:
result = est.evaluate(test_data)
print(f"The Performance of the Model is: {result}")

## Stop SparkContext

In [18]:
from bigdl.orca import stop_orca_context

In [ ]:
stop_orca_context()

-----------------